<a href="https://colab.research.google.com/github/muhozag/sparklearning/blob/linearreg/linreg1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Basics').getOrCreate()
from pyspark.ml.regression import LinearRegression


In [ ]:
data = spark.read.csv('Ecommerce_Customers.csv',inferSchema=True,header=True)

In [ ]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [ ]:
for item in data.head(2)[1]:
    print(item)

hduke@hotmail.com
4547 Archer CommonDiazchester, CA 06566-8576
DarkGreen
31.92627202636016
11.109460728682564
37.268958868297744
2.66403418213262
392.2049334443264


In [ ]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [ ]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [ ]:
assembler = VectorAssembler(
    inputCols=["Avg Session Length", "Time on App",
               "Time on Website",'Length of Membership'],
    outputCol="features")

In [ ]:
output =  assembler.transform(data)

In [ ]:
output.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [ ]:
final_data = output.select("features",'Yearly Amount Spent')

In [ ]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [ ]:
training_data,test_data = final_data.randomSplit([0.7,0.3])

In [ ]:
training_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                352|
|   mean|   499.110068740667|
| stddev|  82.39369418268753|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [ ]:
lr = LinearRegression(labelCol='Yearly Amount Spent')

In [ ]:
lr_model = lr.fit(training_data)

In [ ]:
test_results = lr_model.evaluate(test_data)

In [ ]:
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
| 11.022319561792301|
| 10.977235464647379|
|-3.6114032046654643|
|-20.949888983933192|
|  9.972178236643686|
|  4.167894281656231|
| -7.181752971181027|
| -5.514771268476579|
| -5.136423361382981|
|  3.368010313178104|
|-13.956657218725468|
| 18.349333491080642|
|-4.7918821060237065|
| -5.941301878603042|
|  3.164478591792033|
|-17.780947148608732|
|   7.85963095432129|
| -9.262706401137564|
|  1.208248311193529|
|  12.23630600291392|
+-------------------+
only showing top 20 rows



In [ ]:
test_results.rootMeanSquaredError

10.639307785804762

In [ ]:
test_results.r2

0.9778470720394823

In [ ]:
final_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                500|
|   mean|  499.3140382585909|
| stddev|   79.3147815497068|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [ ]:
unlabeled_data = test_data.select('features')

In [ ]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[29.5324289670579...|
|[30.7377203726281...|
|[30.8794843441274...|
|[31.1239743499119...|
|[31.1695067987115...|
|[31.3662121671876...|
|[31.4474464941278...|
|[31.5147378578019...|
|[31.5257524169682...|
|[31.5316044825729...|
|[31.5741380228732...|
|[31.6005122003032...|
|[31.6253601348306...|
|[31.7242025238451...|
|[31.7366356860502...|
|[31.8164283341993...|
|[31.8209982016720...|
|[31.8279790554652...|
|[31.8293464559211...|
|[31.9096268275227...|
+--------------------+
only showing top 20 rows



In [ ]:
predictions = lr_model.transform(unlabeled_data).show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[29.5324289670579...| 397.6180315108352|
|[30.7377203726281...| 450.8035067315825|
|[30.8794843441274...|493.81800318952014|
|[31.1239743499119...|507.89694282369896|
|[31.1695067987115...| 417.3843525656491|
|[31.3662121671876...| 426.4209882748287|
|[31.4474464941278...|425.78449506640504|
|[31.5147378578019...|  495.327259264938|
|[31.5257524169682...|449.10205017126486|
|[31.5316044825729...|433.14759541618446|
|[31.5741380228732...| 558.3659293793123|
|[31.6005122003032...| 460.8235180000163|
|[31.6253601348306...| 381.1287828629479|
|[31.7242025238451...|509.32918916656354|
|[31.7366356860502...| 493.7689676637399|
|[31.8164283341993...| 518.9034386522651|
|[31.8209982016720...|416.81565005889206|
|[31.8279790554652...| 449.2654539480791|
|[31.8293464559211...|383.94408967678146|
|[31.9096268275227...| 551.2097296703253|
+--------------------+------------